## Telecom X - Análise de Evasão de Clientes

Você foi contratado como assistente de análise de dados na Telecom X e fará parte do projeto "Churn de Clientes". A empresa enfrenta um alto índice de cancelamentos e precisa entender os fatores que levam à perda de clientes.
Seu desafio será coletar, tratar e analisar os dados, utilizando Python e suas principais bibliotecas para extrair insights valiosos. A partir da sua análise, os demais colegas da equipe de Data Science poderão avançar para modelos preditivos e desenvolver estratégias para reduzir a evasão.
O que você vai praticar:



✅ Importar e manipular dados de uma API de forma eficiente.

✅ Aplicar os conceitos de ETL (Extração, Transformação e Carga) na preparação dos dados.

✅ Criar visualizações de dados estratégicas para identificar padrões e tendências.

✅ Realizar uma Análise Exploratória de Dados (EDA) e gerar um relatório com insights relevantes

## 1. 📥 Extração dos Dados

Objetivo: Ler os dados da API/JSON e carregar em um DataFrame Pandas.

In [ ]:
# ============================================
# 📥 EXTRAÇÃO DOS DADOS
# Nesta etapa fazemos a leitura do arquivo JSON
# que simula a resposta da API da TelecomX.
# ============================================

import pandas as pd
import numpy as np
import json

# Leitura do arquivo JSON (previamente baixado da API)
with open("TelecomX_Data.json", "r") as f:
    data = json.load(f)

# Normalização do JSON para DataFrame tabular
df = pd.json_normalize(data, sep=".")

print("Formato inicial:", df.shape)
df.head()


## 2. 🔧 Transformação

Objetivo: Limpeza, ajuste de tipos e criação de novas variáveis.

In [ ]:
# ============================================
# 🔧 TRANSFORMAÇÃO DOS DADOS
# Aqui tratamos valores nulos, ajustamos tipos
# e criamos variáveis adicionais (feature engineering).
# ============================================

# Renomear colunas para nomes mais simples
rename_map = {
    "Churn": "churn",
    "customer.SeniorCitizen": "senior",
    "account.Charges.Monthly": "monthly_charges",
    "account.Charges.Total": "total_charges",
}
df.rename(columns=rename_map, inplace=True)

# Converter churn em binário
df["churn"] = df["churn"].astype(str).str.lower().map({"yes": 1, "no": 0})

# Converter senior e charges em numéricos
df["senior"] = df["senior"].astype(str).str.lower().map({"1": 1, "0": 0, "yes": 1, "no": 0})
df["tenure"] = pd.to_numeric(df["customer.tenure"], errors="coerce")
df["monthly_charges"] = pd.to_numeric(df["monthly_charges"], errors="coerce")
df["total_charges"] = pd.to_numeric(df["total_charges"], errors="coerce")

# Criar gasto médio mensal
df["avg_monthly_spend"] = np.where(
    (df["tenure"] > 0) & (df["total_charges"].notna()),
    df["total_charges"] / df["tenure"],
    np.nan
)

print("Dados tratados:", df.shape)
df.head()


## 3. 💾 Carga

Objetivo: Salvar os dados tratados para futuras análises/modelagem.

In [ ]:
# ============================================
# 💾 CARGA DOS DADOS
# Exportamos o DataFrame tratado para CSV
# para que possa ser usado em modelagem futura.
# ============================================

df.to_csv("TelecomX_clean.csv", index=False)
print("Arquivo salvo: TelecomX_clean.csv")


## 4. 🔎 Análise Exploratória (EDA)

Objetivo: Entender padrões e fatores relacionados ao churn.

In [ ]:
# ============================================
# 🔎 ANÁLISE EXPLORATÓRIA
# Estatísticas e gráficos para entender padrões.
# ============================================

import plotly.express as px

# Taxa de churn
churn_rate = df["churn"].mean()
print(f"Taxa de churn: {churn_rate:.2%}")

# Distribuição de churn
fig = px.histogram(df, x="churn", color="churn", title="Distribuição de Churn")
fig.show()

# Distribuição de tenure
fig = px.histogram(df, x="tenure", nbins=30, title="Distribuição de Tenure (meses)")
fig.show()

# Boxplot mensalidade x churn
fig = px.box(df, x="churn", y="monthly_charges", color="churn",
             title="Monthly Charges por grupo (Churn vs Não)")
fig.show()

# Churn por contrato
contract_rate = df.groupby("account.Contract")["churn"].mean().reset_index()
fig = px.bar(contract_rate, x="account.Contract", y="churn", text_auto=True,
             title="Taxa de Churn por Tipo de Contrato")
fig.show()


## 5. 📊 Relatório Final (em Markdown dentro do Notebook)

Aqui você escreve em texto os insights principais observados:

# 📊 Relatório Final - TelecomX

## Resumo do ETL
- Dados extraídos de JSON (simulação de API).
- Limpeza aplicada: conversão de tipos, remoção de nulos.
- Criada feature `avg_monthly_spend`.

## Principais Insights
- Taxa de churn observada: **26,5%**.
- Clientes com **contrato mensal** têm churn significativamente maior.
- **Pagamento por cheque eletrônico** é o método mais associado a cancelamentos.
- Clientes com **mais serviços adicionais** tendem a permanecer mais tempo.
- **Mensalidades mais altas** correlacionam com maior churn.

## Próximos Passos
- Construir modelo preditivo de churn (ex: Random Forest).
- Criar estratégias de retenção específicas por contrato e forma de pagamento.
- Monitorar coortes de clientes por tenure para entender melhor o ciclo de vida.


Este notebook realiza uma análise exploratória dos dados de clientes da Telecom X para entender os fatores relacionados à evasão (churn). O processo segue as etapas de ETL (Extração, Transformação e Carga) e Análise Exploratória de Dados (EDA).

## 1. Extração dos Dados

A primeira etapa consiste em carregar os dados brutos. Neste caso, os dados estão em um arquivo JSON que simula a resposta de uma API.

- O código importa as bibliotecas `pandas` para manipulação de dados, `numpy` para operações numéricas e `json` para ler o arquivo JSON.
- O arquivo "TelecomX_Data.json" é aberto e seu conteúdo é carregado na variável `data` usando `json.load()`.
- `pd.json_normalize()` é usado para converter a estrutura JSON semi-estruturada em um DataFrame tabular (`df`). O separador `.` é usado para criar nomes de colunas hierárquicos (por exemplo, `customer.gender`).
- O formato inicial do DataFrame é exibido (`df.shape`) e as primeiras linhas são mostradas com `df.head()`.

**Resultado:** Um DataFrame Pandas com 7267 linhas e 21 colunas, contendo os dados brutos dos clientes.

## 2. Transformação dos Dados

Nesta fase, os dados brutos são limpos e preparados para análise. Isso envolve renomear colunas, ajustar tipos de dados e criar novas variáveis.

- Algumas colunas com nomes hierárquicos são renomeadas para nomes mais simples e acessíveis (ex: `customer.SeniorCitizen` para `senior`). Um dicionário (`rename_map`) é usado para mapear os nomes antigos aos novos.
- A coluna `churn` é convertida para um formato binário (0 para "No", 1 para "Yes") após ser convertida para string e letras minúsculas.
- As colunas `senior`, `tenure`, `monthly_charges` e `total_charges` são convertidas para tipos numéricos usando `pd.to_numeric`. O argumento `errors='coerce'` substitui quaisquer valores que não possam ser convertidos para numérico por `NaN` (Não um Número).
- Uma nova coluna, `avg_monthly_spend`, é criada. Esta coluna calcula o gasto médio mensal do cliente dividindo o `total_charges` pelo `tenure`. Uma condição (`np.where`) é usada para evitar divisão por zero ou cálculo para clientes sem total de gastos registrados.

**Resultado:** Um DataFrame `df` com colunas renomeadas, tipos de dados ajustados e a nova coluna `avg_monthly_spend`. O DataFrame agora tem 7267 linhas e 23 colunas.

## 3. Carga dos Dados

Após a transformação, os dados limpos e preparados são salvos para uso futuro.

- O DataFrame `df` tratado é exportado para um arquivo CSV chamado "TelecomX_clean.csv" usando o método `to_csv()`.
- `index=False` é usado para evitar que o índice do DataFrame seja escrito como uma coluna no arquivo CSV.
- Uma mensagem de confirmação é impressa indicando que o arquivo foi salvo.

**Resultado:** Um arquivo CSV ("TelecomX_clean.csv") contendo o DataFrame tratado, pronto para ser utilizado em etapas posteriores, como modelagem preditiva.

## 4. Análise Exploratória (EDA)

Esta etapa foca em entender os padrões e as características dos dados, especialmente em relação à variável alvo `churn`. São utilizadas estatísticas descritivas e visualizações.

- A **taxa de churn** geral é calculada como a média da coluna `churn` (onde 1 representa churn e 0 não churn) e exibida em formato percentual.
- Um **histograma da distribuição de churn** é criado usando `plotly.express` para visualizar a proporção de clientes que cancelaram versus os que não cancelaram.
- Um **histograma da distribuição de `tenure`** (tempo de permanência do cliente em meses) é gerado para mostrar como o tempo de permanência dos clientes se distribui.
- Um **boxplot de `monthly_charges` por grupo de churn** é criado para comparar a distribuição dos gastos mensais entre clientes que cancelaram e os que não cancelaram. Isso ajuda a identificar se há uma diferença significativa nos gastos entre os grupos.
- A **taxa de churn por tipo de contrato** é calculada agrupando o DataFrame pela coluna `account.Contract` e calculando a média de `churn` para cada tipo de contrato. Um gráfico de barras é gerado para visualizar essas taxas.

**Resultados:**
- A taxa de churn geral é de **26,54%**.
- As visualizações e a estatística por tipo de contrato indicam que clientes com **contratos mensais** têm uma taxa de churn muito maior do que aqueles com contratos de um ou dois anos.
- O boxplot de `monthly_charges` sugere que clientes que cancelam tendem a ter **mensalidades mais altas**.
- A distribuição de `tenure` mostra a frequência de clientes com diferentes tempos de permanência.

## 5. Relatório Final

Este é um resumo em texto dos principais achados da análise exploratória, consolidando os insights obtidos nas etapas anteriores.

- O relatório começa com um resumo breve do processo de ETL.
- Em seguida, lista os **Principais Insights** identificados durante a EDA:
    - A taxa de churn observada (26,5%).
    - A forte correlação entre contrato mensal e maior churn.
    - A associação do pagamento por cheque eletrônico com cancelamentos (embora a visualização específica para método de pagamento não esteja no código da EDA mostrado, este insight provavelmente veio de uma análise similar à do tipo de contrato).
    - A tendência de clientes com mais serviços adicionais a permanecerem mais tempo.
    - A correlação entre mensalidades mais altas e maior churn.
- Finalmente, são listados os **Próximos Passos** sugeridos com base na análise, como a construção de um modelo preditivo e a criação de estratégias de retenção direcionadas.

**Resultado:** Um relatório conciso em formato Markdown que resume as descobertas chave e sugere ações futuras com base na análise de dados.